In [1]:
import numpy as np
import pandas as pd
import keras
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense

In [2]:
import common.constant as const
from gomoku.player import Player
from gomoku.gomoku import Gomoku
from common.exception import AlreadyPlacedExcpetion

pygame 2.0.0 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Initialising the ANN
model = Sequential()

model.add(Dense(units = 512, kernel_initializer = 'uniform', activation = 'relu', input_dim = 675))
# model.add(Dropout(rate = 0.1))

model.add(Dense(units = 384, kernel_initializer = 'uniform', activation = 'relu'))
# model.add(Dropout(rate = 0.1))

model.add(Dense(units = 256, kernel_initializer = 'uniform', activation = 'relu'))
# model.add(Dropout(rate = 0.1))

model.add(Dense(units = 225, kernel_initializer = 'uniform', activation = 'softmax'))

opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = opt)

In [4]:
df = pd.DataFrame(columns = ['iteration', 'max_step', 'black_win_rate', 'white_win_rate', 'valid_click_rate'])

In [ ]:
now = datetime.now()
t_str = now.strftime("%Y-%m-%d %H-%M-%S")

max_iter = 10000
max_turn = 10000

black_win_count = 0
white_win_count = 0

verbose = 0

for iteration in range(1, 10000):
    
    gomoku = Gomoku()
    player1 = Player('Player 1', 'B')
    player2 = Player('Player 2', 'W')
    gomoku.start(player1, player2, render = False)
    
    print('Iteration: {}/{}:'.format(iteration, max_iter))

    gomoku.restart()
    
    turn = 'B'
    player1.place(7, 7)
    turn = 'W'
    success = 0
    valid_click_rate = 0


    for i in range(1, max_turn):
        if turn == 'B':
            curr_player = player1
            opponent_player = player2
            
        else:
            curr_player = player2
            opponent_player = player1
        
        X = curr_player.map_my_pieces.reshape((1, 225))
        X = np.append(X, opponent_player.map_my_pieces.reshape((1, 225)), axis = 1)
        X = np.append(X, curr_player.map_prev_move.reshape((1, 225)), axis = 1)
        
        y_pred = model.predict(X)
        
        # pos = y_pred.argmax()
        pos = y_pred.argmax()
        r = pos // 15
        c = pos % 15
        
        win = False
        
        try:
            result = curr_player.place(r, c)
            success = success + 1
            
            if result == const.PLACE_SUCCESS:
                turn = opponent_player.color
                
            elif result == const.PLACE_WIN:
                y_reward = curr_player.map_prev_move.reshape((1, 225))
                model.fit(X, y_reward, verbose = verbose)
                
                if curr_player.color == 'B':
                    black_win_count = black_win_count + 1
                else:
                    white_win_count = white_win_count + 1
                
                win = True
                
        except AlreadyPlacedExcpetion:
            y_reward = curr_player.map_prev_move.reshape((1, 225)) * -1
            model.fit(X, y_reward, verbose = verbose)
            
        if i % 10 == 0:
            valid_click_rate = success / i
            print(' ' * 100, end = '\r')
            print('{}/{} Step: {} Current attempt: ({}, {}) Valid click rate: {:.2f}'.format(i, max_turn, success, r, c, valid_click_rate), end = '\r')
            
        if win:
            break
    
    print(' ' * 100, end = '\r')
    print('{} won! Max step: {} Last attempt: ({}, {}) Valid click rate: {:.2f}'.format(curr_player.name, success, r, c, valid_click_rate))
    
    df.at[iteration - 1, 'iteration'] = iteration
    df.at[iteration - 1, 'max_step'] = success
    df.at[iteration - 1, 'black_win_rate'] = black_win_count / iteration
    df.at[iteration - 1, 'white_win_rate'] = white_win_count / iteration
    df.at[iteration - 1, 'valid_click_rate'] = valid_click_rate
    
    gomoku.save('{} {}'.format(t_str, iteration))
    
    if iteration % 25 == 0:
        df.to_csv('./statistics/{}.csv'.format(t_str))
        
    del gomoku
    del player1
    del player2

Iteration: 1/10000:
Player 2 won! Max step: 83 Last attempt: (4, 7) Valid click rate: 0.49                              
Iteration: 2/10000:
Player 1 won! Max step: 146 Last attempt: (12, 7) Valid click rate: 0.50                            
Iteration: 3/10000:
Player 2 won! Max step: 91 Last attempt: (11, 3) Valid click rate: 0.50                             
Iteration: 4/10000:
Player 2 won! Max step: 77 Last attempt: (9, 12) Valid click rate: 0.50                             
Iteration: 5/10000:
Player 2 won! Max step: 93 Last attempt: (12, 2) Valid click rate: 0.50                             
Iteration: 6/10000:
Player 1 won! Max step: 96 Last attempt: (4, 4) Valid click rate: 0.50                              
Iteration: 7/10000:
Player 1 won! Max step: 98 Last attempt: (12, 7) Valid click rate: 0.50                             
Iteration: 8/10000:
Player 2 won! Max step: 109 Last attempt: (9, 10) Valid click rate: 0.50                            
Iteration: 9/10000:
Player 1 won

Player 2 won! Max step: 107 Last attempt: (10, 2) Valid click rate: 0.47                            
Iteration: 69/10000:
Player 2 won! Max step: 149 Last attempt: (6, 5) Valid click rate: 0.46                             
Iteration: 70/10000:
Player 1 won! Max step: 98 Last attempt: (8, 8) Valid click rate: 0.45                              
Iteration: 71/10000:
Player 2 won! Max step: 145 Last attempt: (8, 5) Valid click rate: 0.44                             
Iteration: 72/10000:
Player 2 won! Max step: 91 Last attempt: (8, 2) Valid click rate: 0.46                              
Iteration: 73/10000:
Player 2 won! Max step: 103 Last attempt: (5, 7) Valid click rate: 0.46                             
Iteration: 74/10000:
Player 1 won! Max step: 104 Last attempt: (9, 6) Valid click rate: 0.46                             
Iteration: 75/10000:
Player 1 won! Max step: 68 Last attempt: (8, 7) Valid click rate: 0.48                              
Iteration: 76/10000:
Player 2 won! Max step: 

Player 2 won! Max step: 89 Last attempt: (6, 5) Valid click rate: 0.47                              
Iteration: 136/10000:
Player 2 won! Max step: 91 Last attempt: (5, 8) Valid click rate: 0.46                              
Iteration: 137/10000:
Player 1 won! Max step: 132 Last attempt: (12, 9) Valid click rate: 0.46                            
Iteration: 138/10000:
Player 2 won! Max step: 75 Last attempt: (7, 5) Valid click rate: 0.46                              
Iteration: 139/10000:
Player 1 won! Max step: 110 Last attempt: (12, 11) Valid click rate: 0.45                           
Iteration: 140/10000:
Player 1 won! Max step: 138 Last attempt: (10, 7) Valid click rate: 0.45                            
Iteration: 141/10000:
Player 1 won! Max step: 144 Last attempt: (14, 7) Valid click rate: 0.44                            
Iteration: 142/10000:
Player 1 won! Max step: 140 Last attempt: (11, 3) Valid click rate: 0.43                            
Iteration: 143/10000:
Player 2 won! Ma

Player 2 won! Max step: 123 Last attempt: (9, 11) Valid click rate: 0.45                            
Iteration: 202/10000:
Player 2 won! Max step: 73 Last attempt: (12, 8) Valid click rate: 0.47                             
Iteration: 203/10000:
Player 1 won! Max step: 126 Last attempt: (9, 8) Valid click rate: 0.45                             
Iteration: 204/10000:
Player 2 won! Max step: 119 Last attempt: (1, 12) Valid click rate: 0.45                            
Iteration: 205/10000:
Player 2 won! Max step: 121 Last attempt: (0, 5) Valid click rate: 0.44                             
Iteration: 206/10000:
Player 1 won! Max step: 88 Last attempt: (6, 1) Valid click rate: 0.46                              
Iteration: 207/10000:
Player 1 won! Max step: 100 Last attempt: (13, 5) Valid click rate: 0.45                            
Iteration: 208/10000:
Player 2 won! Max step: 117 Last attempt: (12, 12) Valid click rate: 0.45                           
Iteration: 209/10000:
Player 1 won! Ma

Player 2 won! Max step: 105 Last attempt: (7, 10) Valid click rate: 0.44                            
Iteration: 268/10000:
Player 1 won! Max step: 82 Last attempt: (0, 2) Valid click rate: 0.46                              
Iteration: 269/10000:
Player 1 won! Max step: 100 Last attempt: (6, 1) Valid click rate: 0.46                             
Iteration: 270/10000:
Player 2 won! Max step: 83 Last attempt: (3, 1) Valid click rate: 0.46                              
Iteration: 271/10000:
Player 2 won! Max step: 127 Last attempt: (5, 5) Valid click rate: 0.46                             
Iteration: 272/10000:
Player 1 won! Max step: 108 Last attempt: (4, 11) Valid click rate: 0.46                            
Iteration: 273/10000:
Player 2 won! Max step: 131 Last attempt: (1, 2) Valid click rate: 0.44                             
Iteration: 274/10000:
Player 1 won! Max step: 90 Last attempt: (4, 9) Valid click rate: 0.45                              
Iteration: 275/10000:
Player 1 won! Ma

...

Player 1 won! Max step: 118 Last attempt: (11, 13) Valid click rate: 0.43                           
Iteration: 5152/10000:
Player 1 won! Max step: 110 Last attempt: (6, 1) Valid click rate: 0.45                             
Iteration: 5153/10000:
Player 1 won! Max step: 108 Last attempt: (9, 10) Valid click rate: 0.45                            
Iteration: 5154/10000:
Player 2 won! Max step: 85 Last attempt: (4, 7) Valid click rate: 0.45                              
Iteration: 5155/10000:
Player 1 won! Max step: 140 Last attempt: (11, 6) Valid click rate: 0.46                            
Iteration: 5156/10000:
Player 1 won! Max step: 152 Last attempt: (13, 5) Valid click rate: 0.44                            
Iteration: 5157/10000:
Player 1 won! Max step: 50 Last attempt: (8, 1) Valid click rate: 0.45                              
Iteration: 5158/10000:
Player 1 won! Max step: 82 Last attempt: (4, 11) Valid click rate: 0.46                             
Iteration: 5159/10000:
Player 1

Player 1 won! Max step: 76 Last attempt: (12, 11) Valid click rate: 0.45                            
Iteration: 5218/10000:
Player 2 won! Max step: 113 Last attempt: (1, 3) Valid click rate: 0.46                             
Iteration: 5219/10000:
Player 1 won! Max step: 100 Last attempt: (8, 3) Valid click rate: 0.46                             
Iteration: 5220/10000:
Player 1 won! Max step: 80 Last attempt: (7, 12) Valid click rate: 0.47                             
Iteration: 5221/10000:
Player 2 won! Max step: 129 Last attempt: (1, 12) Valid click rate: 0.46                            
Iteration: 5222/10000:
Player 1 won! Max step: 138 Last attempt: (2, 6) Valid click rate: 0.45                             
Iteration: 5223/10000:
Player 1 won! Max step: 126 Last attempt: (6, 5) Valid click rate: 0.43                             
Iteration: 5224/10000:
Player 2 won! Max step: 135 Last attempt: (6, 4) Valid click rate: 0.44                             
Iteration: 5225/10000:
Player 2

Player 2 won! Max step: 109 Last attempt: (8, 12) Valid click rate: 0.46                            
Iteration: 5284/10000:
Player 1 won! Max step: 106 Last attempt: (11, 7) Valid click rate: 0.47                            
Iteration: 5285/10000:
Player 1 won! Max step: 58 Last attempt: (8, 11) Valid click rate: 0.47                             
Iteration: 5286/10000:
Player 1 won! Max step: 144 Last attempt: (7, 14) Valid click rate: 0.45                            
Iteration: 5287/10000:
Player 1 won! Max step: 104 Last attempt: (12, 2) Valid click rate: 0.45                            
Iteration: 5288/10000:
Player 2 won! Max step: 117 Last attempt: (2, 9) Valid click rate: 0.44                             
Iteration: 5289/10000:
Player 1 won! Max step: 72 Last attempt: (8, 9) Valid click rate: 0.47                              
Iteration: 5290/10000:
Player 1 won! Max step: 152 Last attempt: (12, 7) Valid click rate: 0.45                            
Iteration: 5291/10000:
Player 1

Player 2 won! Max step: 119 Last attempt: (4, 5) Valid click rate: 0.46                             
Iteration: 5350/10000:
Player 1 won! Max step: 108 Last attempt: (10, 4) Valid click rate: 0.45                            
Iteration: 5351/10000:
Player 2 won! Max step: 109 Last attempt: (6, 0) Valid click rate: 0.45                             
Iteration: 5352/10000:
Player 2 won! Max step: 125 Last attempt: (3, 4) Valid click rate: 0.45                             
Iteration: 5353/10000:
Player 2 won! Max step: 157 Last attempt: (9, 8) Valid click rate: 0.44                             
Iteration: 5354/10000:
Player 2 won! Max step: 105 Last attempt: (3, 10) Valid click rate: 0.44                            
Iteration: 5355/10000:
Player 2 won! Max step: 157 Last attempt: (7, 9) Valid click rate: 0.43                             
Iteration: 5356/10000:
Player 1 won! Max step: 116 Last attempt: (3, 7) Valid click rate: 0.44                             
Iteration: 5357/10000:
Player 1

Player 1 won! Max step: 122 Last attempt: (11, 3) Valid click rate: 0.45                            
Iteration: 5416/10000:
Player 1 won! Max step: 106 Last attempt: (0, 10) Valid click rate: 0.45                            
Iteration: 5417/10000:
Player 1 won! Max step: 130 Last attempt: (6, 6) Valid click rate: 0.45                             
Iteration: 5418/10000:
Player 2 won! Max step: 85 Last attempt: (7, 1) Valid click rate: 0.46                              
Iteration: 5419/10000:
Player 1 won! Max step: 88 Last attempt: (4, 4) Valid click rate: 0.46                              
Iteration: 5420/10000:
Player 2 won! Max step: 79 Last attempt: (13, 3) Valid click rate: 0.47                             
Iteration: 5421/10000:
Player 1 won! Max step: 98 Last attempt: (14, 14) Valid click rate: 0.46                            
Iteration: 5422/10000:
Player 1 won! Max step: 100 Last attempt: (9, 6) Valid click rate: 0.47                             
Iteration: 5423/10000:
Player 1

In [ ]:
gomoku.save()

In [ ]:
df.head(5)

In [ ]:
import concurrent.futures

def foo(bar):
    print('hello {}'.format(bar))
    return 'foo'

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(foo, 'world!')
    future2 = executor.submit(foo, 'world2!')
    return_value = future.result()
    print(return_value)
    return_value2 = future2.result()
    print(return_value2)